# Import Libraries

In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup
from time import sleep
import sqlite3
from sqlite3 import Error

# Setup Variables

In [2]:
profile_counter = 0
DB_PATH = r".\pythonsqlite.db"
CHROME_DRIVER_PATH = r"C:\Program Files (x86)\chromedriver.exe" 
CONNECTIONS_LINK = "https://www.linkedin.com/mynetwork/invite-connect/connections/"
LINKEDIN_LINK = "https://www.linkedin.com"
LOG_PATH = r'.\linkedin.log'
SCROLL_PAUSE_TIME = 1
USERNAME = 'hamidreza.samsami@gmail.com'
PASSWORD = '$C5ysp6BTbEacK_'

# SQLite Functions

In [3]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [4]:
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [5]:
def create_linkedin_profiles_table(conn):
    sql_create_linkedin_profiles_table = """ CREATE TABLE IF NOT EXISTS linkedin_profiles (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        headline text
                                    ); """
    if conn is not None:
        create_table(conn, sql_create_linkedin_profiles_table)

    else:
        print("Error! cannot create the database connection.")

In [6]:
def insert_to_db(conn, profile):
    sql = ''' INSERT INTO linkedin_profiles(name,headline)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, profile)
    conn.commit()
    return cur.lastrowid

In [7]:
def close_connection(conn):
    conn.close()

# Logfile

In [8]:
def open_logfile(path):
    return open(path,'a',encoding='utf-8')

def insert_to_logfile(file,profile):
    global profile_counter
    profile_counter += 1
    file.write(f"{profile_counter},\"{profile[0]}\",\"{profile[1]}\"\n")
        
def close_logfile(file):
    file.close()

# Chrome Functions

In [9]:
def create_web_driver(PATH):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    return webdriver.Chrome(executable_path=PATH,options=options)

In [10]:
def parse_current_page():
    return BeautifulSoup(driver.page_source, "html.parser")

In [11]:
def get_scroll_height():
    return driver.execute_script("return document.body.scrollHeight")

In [12]:
def scroll_down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Linkedin Functions

In [13]:
def login_to_linkedin():
    driver.get(LINKEDIN_LINK)
    driver.implicitly_wait(10)
    username = driver.find_element_by_class_name('input__input')
    username.send_keys(USERNAME)
    password = driver.find_element_by_id('session_password')
    password.send_keys(PASSWORD) 
    log_in_button = driver.find_element_by_class_name('sign-in-form__submit-button') 
    log_in_button.click()

In [14]:
def get_my_profile_info(soupified):
    my_name = soupified.find("img", {"class": "global-nav__me-photo"})['alt'].strip()
    my_headline = soupified.find("p", {"class": "identity-headline"}).get_text().strip()
    return (my_name,my_headline)

In [15]:
def get_connection_profile_info(item):
    name = item.find("span",{"class":"mn-connection-card__name"}).text.strip()
    headline = item.find("span",{"class":"mn-connection-card__occupation"}).text.strip()
    return (name, headline)

In [16]:
def insert_profile(profile):
    insert_to_logfile(logfile,profile)
    insert_to_db(conn,profile)

In [17]:
def load_connections_page():
    driver.get(CONNECTIONS_LINK)

    last_height = get_scroll_height()
    new_height = 0

    while True:
        scroll_down()
        sleep(SCROLL_PAUSE_TIME)
        new_height = get_scroll_height()

        if new_height == last_height:
            try:
                driver.find_element_by_xpath("//*[text() = 'Show more results']").click()
                sleep(SCROLL_PAUSE_TIME)
            except:
                break

        last_height = new_height

# Main Process

###  Prepare DB, Logfile and Web driver

In [18]:
conn = create_connection(DB_PATH)
create_linkedin_profiles_table(conn)
driver = create_web_driver(CHROME_DRIVER_PATH)
logfile = open_logfile(LOG_PATH)

###  Login

In [19]:
login_to_linkedin()

### Extract and Store My Info

In [20]:
page = parse_current_page()
my_profile = get_my_profile_info(page)
insert_profile(my_profile)

### Extract and Store My Connections info

In [21]:
load_connections_page()
page = parse_current_page()
all_connections = page.find_all("a", {"class": "mn-connection-card__link"}) 
for item in all_connections:
    profile = get_connection_profile_info(item)
    insert_profile(profile)

# End Process

In [22]:
close_connection(conn)
close_logfile(logfile)